In [1]:
import os
import sys
import time
from datetime import datetime, timedelta
import wget
import sys
from ftplib import FTP
from multiprocessing import Pool
import glob
import get_gfs_parallel as get_gfs


wrf_dir = '/home/peter/WRF_MODEL'
run = '18'
hours = 39
delay = 11

sleep = 120

In [4]:
date = datetime.today() - timedelta(days=5)

In [6]:
date_string = date.strftime('%Y%m%d')
year = date.strftime('%Y')
month = date.strftime('%m')
day = date.strftime('%d')

In [7]:
start = datetime(int(year), int(month), int(day), hour=int(run), minute=0, second=0)
start = start + timedelta(hours=delay)
end = start + timedelta(hours=hours)

In [ ]:
#GET_DATA
names=[]
for i in range(delay,int(delay+hours)+1):
    names.append('gfs.t'+str(run)+'z.pgrb2.0p25.f'+str(i).zfill(3))


get_gfs.is_it_runtime(date_string,run)

link = str("https://nomads.ncep.noaa.gov/pub/data/nccf/com/gfs/prod/gfs."+date_string+"/"+str(run)+"/atmos/")
data_dir = "/mnt/C6ECF47AECF46659/DATA_2/"

pull = get_gfs.pullable_files(date_string,run,names)
while True:
    if len(pull) == (hours+1):
        break
    else:
        print('sleep')
        time.sleep(1200)
        pull = get_gfs.pullable_files(date_string,run,names)



links = [link+name for name in pull]
file_dest = [data_dir+name for name in pull]

if __name__ == '__main__':
    start_time = time.time()
    value = datetime.fromtimestamp(start_time)
    print(value.strftime('%Y-%m-%d %H:%M:%S'))
    pool = Pool(processes=15)
    pooled = [pool.apply_async(get_gfs.run_wget, args=(link,file)) for link, file in zip(links,file_dest)]
    TIMEOUT = sleep * (hours+1)
    time.sleep(TIMEOUT)
    down = get_gfs.get_downloaded(data_dir)
    if len(down) == hours+1:
        pool.terminate()
    else:
        time.sleep(300)
        if len(get_gfs.get_downloaded()) > len(down):
            print('still working be patient')
            time.sleep(sleep * ((hours+1) - len(get_gfs.get_downloaded()))) #still woring be patient
            pool.terminate()
        if len(get_gfs.get_downloaded()) < hours+1:
            while True:             #never ending retry loop
                pool.terminate()
                sleep = sleep + 30
                print('attempt: ' , ((sleep - 120) / 30))
                get_gfs.try_again(down, sleep, file_dest)
                down = get_gfs.get_downloaded()
                if len(down) == hours+1:
                    pool.terminate()
                    break
                elif ((sleep - 120) / 30) == 3: #break after 3 attempts
                    break
        else:
            pass




down = [down for down in glob.glob(data_dir + "*") if down[-1] != 'p']
print('pulled' , len(down), ' of ', hours+1, 'total after ' , round((time.time() - start_time)/3600, 2) , 'hours')


get runs
check
2022-03-30 15:42:20
